# Load

In [1]:
# --- Setup
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

# --- Paths (adjust if needed)
DATA_DIR = Path("./data/bronze")  # folder where your CSVs live

# Filenames (rename if yours differ)
f_accounts      = DATA_DIR / "ravenstack_accounts.csv"
f_subs          = DATA_DIR / "ravenstack_subscriptions.csv"
f_usage         = DATA_DIR / "ravenstack_feature_usage.csv"
f_tickets       = DATA_DIR / "ravenstack_support_tickets.csv"
f_churn         = DATA_DIR / "ravenstack_churn_events.csv"


In [2]:
# --- Helper to read with parsed dates and lowercased columns
def read_csv_with_dates(path, parse_cols=None):
    df = pd.read_csv(path)
    df.columns = [c.strip().lower() for c in df.columns]
    if parse_cols:
        for c in parse_cols:
            if c in df.columns:
                df[c] = pd.to_datetime(df[c], errors="coerce", utc=False)
    return df

accounts = read_csv_with_dates(f_accounts, parse_cols=["signup_date"])
subs     = read_csv_with_dates(f_subs,     parse_cols=["start_date","end_date"])
usage    = read_csv_with_dates(f_usage,    parse_cols=["usage_date"])
tickets  = read_csv_with_dates(f_tickets,  parse_cols=["submitted_at","closed_at"])
churn    = read_csv_with_dates(f_churn,    parse_cols=["churn_date"])

# --- Basic shapes and periods
def table_summary(name, df, date_cols=()):
    print(f"\n[{name}] rows={len(df):,} cols={df.shape[1]}")
    for c in date_cols:
        if c in df.columns:
            print(f"  {c}: {df[c].min()}  →  {df[c].max()}")
    print("  Nulls (% top 3):")
    nulls = (df.isna().mean()*100).sort_values(ascending=False).head(3).round(2)
    print(nulls.to_string())

table_summary("accounts", accounts, ["signup_date"])
table_summary("subscriptions", subs, ["start_date","end_date"])
table_summary("feature_usage", usage, ["usage_date"])
table_summary("support_tickets", tickets, ["submitted_at","closed_at"])
table_summary("churn_events", churn, ["churn_date"])



[accounts] rows=500 cols=10
  signup_date: 2023-01-02 00:00:00  →  2024-12-31 00:00:00
  Nulls (% top 3):
account_id      0.0
account_name    0.0
industry        0.0

[subscriptions] rows=5,000 cols=14
  start_date: 2023-01-09 00:00:00  →  2024-12-31 00:00:00
  end_date: 2023-04-05 00:00:00  →  2024-12-31 00:00:00
  Nulls (% top 3):
end_date           90.28
subscription_id     0.00
account_id          0.00

[feature_usage] rows=25,000 cols=8
  usage_date: 2023-01-01 00:00:00  →  2024-12-31 00:00:00
  Nulls (% top 3):
usage_id           0.0
subscription_id    0.0
usage_date         0.0

[support_tickets] rows=2,000 cols=9
  submitted_at: 2023-01-02 00:00:00  →  2024-12-31 00:00:00
  closed_at: 2023-01-03 03:00:00  →  2024-12-31 19:00:00
  Nulls (% top 3):
satisfaction_score    41.25
ticket_id              0.00
account_id             0.00

[churn_events] rows=600 cols=9
  churn_date: 2023-01-25 00:00:00  →  2024-12-31 00:00:00
  Nulls (% top 3):
feedback_text     24.67
churn_event_id   

# EDA

## Modeling

In [3]:
accounts.sample(5)

,account_id,account_name,industry,country,signup_date,referral_source,plan_tier,seats,is_trial,churn_flag
298,A-4e631b,Company_298,FinTech,US,2024-03-16,other,Enterprise,22,False,False
285,A-417d2f,Company_285,Cybersecurity,IN,2023-07-04,partner,Pro,4,True,False
412,A-7920cc,Company_412,Cybersecurity,FR,2023-11-23,ads,Pro,39,False,False
423,A-3cc791,Company_423,EdTech,US,2024-01-10,other,Enterprise,6,False,False
208,A-508199,Company_208,FinTech,AU,2024-01-28,event,Basic,19,False,False


In [4]:
accounts[accounts['account_id'] == 'A-3cc791'] #A-3c1a3f

,account_id,account_name,industry,country,signup_date,referral_source,plan_tier,seats,is_trial,churn_flag
423,A-3cc791,Company_423,EdTech,US,2024-01-10,other,Enterprise,6,False,False


In [5]:
accounts.columns

Index(['account_id', 'account_name', 'industry', 'country', 'signup_date',
       'referral_source', 'plan_tier', 'seats', 'is_trial', 'churn_flag'],
      dtype='object')

In [6]:
subs['start_date'] = pd.to_datetime(subs['start_date'])
subs[subs['account_id'] == 'A-3cc791'].sort_values(by='start_date')

,subscription_id,account_id,start_date,end_date,plan_tier,seats,mrr_amount,arr_amount,is_trial,upgrade_flag,downgrade_flag,churn_flag,billing_frequency,auto_renew_flag
965,S-343642,A-3cc791,2024-01-31,NaT,Basic,25,0,0,True,False,False,False,annual,False
667,S-879b43,A-3cc791,2024-03-26,NaT,Enterprise,6,1194,14328,False,False,False,False,monthly,True
4242,S-6b144c,A-3cc791,2024-04-10,NaT,Basic,16,304,3648,False,False,False,False,annual,True
4216,S-a916c8,A-3cc791,2024-04-26,NaT,Basic,12,0,0,True,False,False,False,monthly,False
3851,S-491d53,A-3cc791,2024-05-23,NaT,Pro,12,588,7056,False,False,False,False,monthly,True
1667,S-f981fe,A-3cc791,2024-06-01,NaT,Enterprise,22,4378,52536,False,False,False,False,monthly,True
2579,S-a924c2,A-3cc791,2024-07-08,NaT,Pro,6,294,3528,False,False,False,False,annual,True
1521,S-dbceba,A-3cc791,2024-12-19,NaT,Pro,36,1764,21168,False,False,False,False,monthly,True


In [7]:
subs[subs['subscription_id'].duplicated(keep=False)].sort_values('subscription_id')

,subscription_id,account_id,start_date,end_date,plan_tier,seats,mrr_amount,arr_amount,is_trial,upgrade_flag,downgrade_flag,churn_flag,billing_frequency,auto_renew_flag


In [8]:
subs.plan_tier.unique()

array(['Enterprise', 'Pro', 'Basic'], dtype=object)

In [13]:
#usage.head(10)
usage['usage_date'] = pd.to_datetime(usage['usage_date'])

usage[usage['subscription_id'] == 'S-343642'].sort_values(by='usage_date')

,usage_id,subscription_id,usage_date,feature_name,usage_count,usage_duration_secs,error_count,is_beta_feature
23463,U-1da969,S-343642,2023-02-20,feature_17,8,3672,2,False
13529,U-cd1444,S-343642,2023-02-21,feature_12,7,392,0,False
6852,U-a5c3a1,S-343642,2023-05-01,feature_12,10,5910,0,False
1056,U-886d7b,S-343642,2023-05-27,feature_16,8,4104,0,False
18517,U-8caf13,S-343642,2023-08-19,feature_3,7,3164,0,True
16645,U-256c78,S-343642,2023-11-29,feature_16,9,3024,0,False
19277,U-9d9226,S-343642,2024-11-25,feature_25,13,2873,0,False


In [10]:
#tickets
#churn

tickets.head(10)

,ticket_id,account_id,submitted_at,closed_at,resolution_time_hours,priority,first_response_time_minutes,satisfaction_score,escalation_flag
0,T-0024de,A-712f1c,2023-07-27,2023-07-28 03:00:00,27.0,high,74,NaN,False
1,T-4d04b9,A-e43bf7,2024-07-08,2024-07-09 03:00:00,27.0,urgent,144,NaN,False
2,T-d5e12f,A-0f3e88,2024-10-17,2024-10-17 19:00:00,19.0,urgent,93,4.0,False
3,T-dfce9a,A-4c56c9,2024-09-08,2024-09-09 23:00:00,47.0,medium,126,5.0,False
4,T-c59f77,A-6f8ad2,2024-11-30,2024-12-01 02:00:00,26.0,medium,8,NaN,False
5,T-90f06d,A-94c3cd,2023-07-27,2023-07-27 09:00:00,9.0,medium,60,NaN,False
6,T-30b537,A-2e4581,2023-09-09,2023-09-10 03:00:00,27.0,urgent,64,3.0,False
7,T-60242d,A-72799b,2024-01-29,2024-01-29 12:00:00,12.0,low,56,4.0,True
8,T-7119c9,A-b179bf,2023-08-27,2023-08-27 16:00:00,16.0,urgent,154,NaN,False
9,T-b0edf2,A-7cfe77,2024-05-06,2024-05-07 04:00:00,28.0,medium,150,3.0,False


In [11]:
# Example (conceptual)
subs['account_id'].nunique(), subs['subscription_id'].nunique()
usage['subscription_id'].nunique()
# sanity check: no orphans
orphans = usage[~usage['subscription_id'].isin(subs['subscription_id'])]
orphans

,usage_id,subscription_id,usage_date,feature_name,usage_count,usage_duration_secs,error_count,is_beta_feature


## TEST KPIs

In [12]:
# --- Parse dates (safeguard) ---
subs['start_date'] = pd.to_datetime(subs['start_date'], errors='coerce')
subs['end_date'] = pd.to_datetime(subs['end_date'], errors='coerce')

# --- Identify trial and paid subs ---
trial_subs = subs[subs['is_trial'] == True].copy()
paid_subs  = subs[subs['is_trial'] == False].copy()

# --- Flag conversions ---
# For each trial account, check if there's a later paid subscription
trial_accounts = trial_subs[['account_id']].drop_duplicates()
paid_accounts  = paid_subs[['account_id']].drop_duplicates()

# Merge to find trial accounts with any paid plan later on
trial_paid = pd.merge(
    trial_subs[['account_id','start_date']].groupby('account_id').min().rename(columns={'start_date':'trial_start'}),
    paid_subs[['account_id','start_date']].groupby('account_id').min().rename(columns={'start_date':'first_paid_start'}),
    on='account_id', how='left'
)
trial_paid['converted_to_paid'] = trial_paid['first_paid_start'] > trial_paid['trial_start']

# --- Compute KPI (exploratory) ---
total_trials = len(trial_paid)
converted_trials = trial_paid['converted_to_paid'].sum()
trial_to_paid_rate = converted_trials / total_trials if total_trials > 0 else None

print(f"Trial accounts: {total_trials}")
print(f"Converted to paid: {converted_trials}")
print(f"Trial-to-Paid Conversion Rate: {trial_to_paid_rate:.2%}" if trial_to_paid_rate is not None else "No trial data")

# --- Preview sample ---
trial_paid.sample(10)

Trial accounts: 403
Converted to paid: 86
Trial-to-Paid Conversion Rate: 21.34%


,trial_start,first_paid_start,converted_to_paid
account_id,,,
A-bc87bc,2024-12-01,2024-09-11,False
A-a8b49c,2023-11-24,2023-10-14,False
A-832ec2,2024-01-02,2024-02-02,True
A-158f4e,2023-08-30,2023-06-02,False
A-2d4502,2024-09-25,2024-04-30,False
A-712f1c,2024-01-28,2024-01-03,False
A-66224b,2024-11-20,2024-10-20,False
A-fce879,2024-10-18,2024-09-28,False
A-075038,2023-11-26,2023-07-14,False
